Author: Soumyadip Mukherjee
Team Name: Soumo275


##Testing againt the train data once then finding the final csv against the test data

Testing it once against the training data to check correct working of the model

In [2]:
import zipfile
import os

zip_path = "soil-classification.zip"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("soil-classification")

In [3]:
import os                     # histogram
import numpy as np
import pandas as pd
import cv2
import joblib

# Paths
IMAGE_FOLDER = '/content/soil-classification/soil_classification-2025/train'
CSV_FILE = '/content/soil-classification/soil_classification-2025/train_labels.csv'
MODEL_FILE = 'soil_rf_model.joblib'
OUTPUT_CSV = '/content/soil-classification/soil_classification-2025/t_input_with_predictions.csv'

# Load model and label encoder
clf, le = joblib.load(MODEL_FILE)
print("Model and LabelEncoder loaded.")

# Load CSV
df = pd.read_csv(CSV_FILE)
predicted_soil_types = []

# Use the same function as in training
def extract_color_histogram(image, bins=(8, 8, 8)):
    # Resize to 224x224 (just like training)
    img_resized = cv2.resize(image, (224, 224))
    # Convert BGR to RGB
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    # Compute 3D color histogram without masking black pixels
    hist = cv2.calcHist([img_rgb], [0, 1, 2], None, bins, [0, 256] * 3)
    hist = cv2.normalize(hist, hist).flatten()
    return hist

for _, row in df.iterrows():
    image_id = row['image_id']
    image_path = os.path.join(IMAGE_FOLDER, image_id)

    img = cv2.imread(image_path)
    if img is None:
        print(f"[WARNING] Cannot read image: {image_path}")
        predicted_soil_types.append("Unknown")
        continue

    hist_features = extract_color_histogram(img, bins=(8, 8, 8))

    # Reshape to 2D array for prediction
    hist_features = hist_features.reshape(1, -1)

    pred_encoded = clf.predict(hist_features)[0]
    pred_label = le.inverse_transform([pred_encoded])[0]

    print(f"Image: {image_id}, Predicted: {pred_label}")
    predicted_soil_types.append(pred_label)

df['predicted_soil_type'] = predicted_soil_types
df.to_csv(OUTPUT_CSV, index=False)
print(f"\nPredictions written to: {OUTPUT_CSV}")

Model and LabelEncoder loaded.
Image: img_ed005410.jpg, Predicted: Alluvial soil
Image: img_0c5ecd2a.jpg, Predicted: Alluvial soil
Image: img_ed713bb5.jpg, Predicted: Alluvial soil
Image: img_12c58874.jpg, Predicted: Alluvial soil
Image: img_eff357af.jpg, Predicted: Alluvial soil
Image: img_ebbd4d97.jpeg, Predicted: Alluvial soil
Image: img_61a4be9b.jpg, Predicted: Alluvial soil
Image: img_f8ee6267.jpg, Predicted: Alluvial soil
Image: img_903747fa.jpg, Predicted: Alluvial soil
Image: img_be9dc251.jpg, Predicted: Alluvial soil
Image: img_47b2a710.jpg, Predicted: Alluvial soil
Image: img_92d7778b.jpg, Predicted: Alluvial soil
Image: img_db1d9b5f.jpg, Predicted: Alluvial soil
Image: img_6030bc4f.jpg, Predicted: Alluvial soil
Image: img_dfcfbfe0.jpg, Predicted: Alluvial soil
Image: img_74efd1ac.jpg, Predicted: Alluvial soil
Image: img_20987a8c.jpg, Predicted: Alluvial soil
Image: img_1f23d3f3.jpg, Predicted: Alluvial soil
Image: img_320c82e1.jpg, Predicted: Alluvial soil
Image: img_6d534f1

checking the mismatched images

In [4]:
import pandas as pd

# CSV paths
csv1 = '/content/soil-classification/soil_classification-2025/train_labels.csv'
csv2 = '/content/soil-classification/soil_classification-2025/t_input_with_predictions.csv'

# Load CSVs
df1 = pd.read_csv(csv1)
df2 = pd.read_csv(csv2)

# Check row count
if len(df1) != len(df2):
    raise ValueError("CSV files have different number of rows")

# Compare predictions
matches = df1['soil_type'] == df2['predicted_soil_type']
percent_match = matches.sum() / len(matches) * 100
print(f"\nPercentage of correct predictions: {percent_match:.2f}%")

# Print mismatched rows
mismatches = df1[~matches].copy()
mismatches['predicted_soil_type'] = df2.loc[~matches, 'predicted_soil_type']

print("\nMismatched Predictions:")
if mismatches.empty:
    print("All predictions matched.")
else:
    print(mismatches[['image_id', 'soil_type', 'predicted_soil_type']])


Percentage of correct predictions: 99.02%

Mismatched Predictions:
              image_id      soil_type predicted_soil_type
309   img_31fafc06.jpg  Alluvial soil          Black Soil
350   img_d4471d29.jpg  Alluvial soil          Black Soil
553   img_bd1bb46a.jpg      Clay soil            Red soil
591   img_b76d9a00.jpg      Clay soil       Alluvial soil
618   img_72f10347.jpg      Clay soil       Alluvial soil
620   img_cf9901a9.jpg      Clay soil       Alluvial soil
651   img_a9eaebac.jpg      Clay soil          Black Soil
701   img_5d65eae8.jpg      Clay soil       Alluvial soil
711   img_7c647425.jpg      Clay soil            Red soil
712   img_ab6e0a7e.jpg      Clay soil          Black Soil
743   img_32af671f.jpg       Red soil          Black Soil
1047  img_fbc9cf4d.jpg     Black Soil       Alluvial soil


creating the final csv with the train data

In [5]:
import os
import numpy as np
import pandas as pd
import cv2
import joblib

# Paths
IMAGE_FOLDER = '/content/soil-classification/soil_classification-2025/test'
CSV_FILE = '/content/soil-classification/soil_classification-2025/test_ids.csv'
MODEL_FILE = 'soil_rf_model.joblib'
OUTPUT_CSV = '/content/soil-classification/soil_classification-2025/test_input_with_predictions.csv'

# Load model and label encoder
clf, le = joblib.load(MODEL_FILE)
print("Model and LabelEncoder loaded.")

# Load test CSV
df = pd.read_csv(CSV_FILE)

# Feature extraction function exactly as used in training
def extract_color_histogram(image, bins=(8, 8, 8)):
    img_resized = cv2.resize(image, (224, 224))
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    hist = cv2.calcHist([img_rgb], [0, 1, 2], None, bins, [0, 256] * 3)
    hist = cv2.normalize(hist, hist).flatten()
    return hist

predicted_soil_types = []

for image_id in df['image_id']:
    image_path = os.path.join(IMAGE_FOLDER, image_id)
    img = cv2.imread(image_path)
    if img is None:
        print(f"[WARNING] Cannot read image: {image_path}")
        predicted_soil_types.append("Unknown")
        continue

    # Extract features the same way as training
    hist_features = extract_color_histogram(img, bins=(8, 8, 8))
    hist_features = hist_features.reshape(1, -1)  # Reshape for prediction

    pred_encoded = clf.predict(hist_features)[0]
    pred_label = le.inverse_transform([pred_encoded])[0]

    print(f"Image: {image_id}, Predicted: {pred_label}")
    predicted_soil_types.append(pred_label)

df['soil_type'] = predicted_soil_types
df.to_csv(OUTPUT_CSV, index=False)
print(f"\nPredictions written to: {OUTPUT_CSV}")

Model and LabelEncoder loaded.
Image: img_cdf80d6f.jpeg, Predicted: Alluvial soil
Image: img_c0142a80.jpg, Predicted: Alluvial soil
Image: img_91168fb0.jpg, Predicted: Alluvial soil
Image: img_9822190f.jpg, Predicted: Alluvial soil
Image: img_e5fc436c.jpeg, Predicted: Alluvial soil
Image: img_3d6e64a1.jpg, Predicted: Alluvial soil
Image: img_ccb81e64.jpeg, Predicted: Alluvial soil
Image: img_9a499fc5.jpeg, Predicted: Alluvial soil
Image: img_b957da08.jpeg, Predicted: Alluvial soil
Image: img_c62bd1a1.jpg, Predicted: Alluvial soil
Image: img_20eba00e.jpg, Predicted: Alluvial soil
Image: img_46a2c139.jpg, Predicted: Alluvial soil
Image: img_f303eb5f.jpg, Predicted: Alluvial soil
Image: img_3ddd1263.jpg, Predicted: Alluvial soil
Image: img_c115ac40.jpg, Predicted: Alluvial soil
Image: img_be60411c.jpeg, Predicted: Alluvial soil
Image: img_887c5e3d.jpg, Predicted: Alluvial soil
Image: img_85ed00e6.jpg, Predicted: Alluvial soil
Image: img_3819f9e3.jpg, Predicted: Alluvial soil
Image: img_18